Ziel: Man gibt dem Model eine 60 Einheiten lange Folge an Marktdaten und dann soll das Model den nächsten Marktpreis berechnen. Also den '61' und es soll halt nur einen Preis/Zahl ausgeben. Was es aber tut ist, dass es wieder 60 Preise ausgibt. Also von 1 bis 61. Ich weiß, dass das mit der seq_length zusammen hängt, leider weiß ich aber nicht wie ich dem Model sagen kann dass es mir nur einen Preis ausgeben soll. Also irgendwie muss ich die seq_length des outputs von 60 auf 1 bekommen.

Meine Bitte wäre ob du mal über mein Model gucken kannst und vielleicht hast du ja eine Lösung für mein Problem. 
Mit freundlichen Grüßen
Niklas 

In [6]:
#Imports
from os import listdir
import random
import torch.optim as optim
from torch.autograd import Variable
import torch.nn.functional as F
import torch.nn as nn
import torch
from torch.nn.modules import loss
import math
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import matplotlib.pyplot as plt

In [7]:
def getDataFromCSV(path):
    data = pd.read_csv(path, index_col="Date", parse_dates=True)
    return data

df = getDataFromCSV(r"C:\Users\jensb\Documents\Praktikum\Stock_Prediction\03-07-2021\JugendForscht\CSVs\SPX.csv")
df

,OPEN,HIGH,LOW,CLOSE
Date,,,,
2015-01-02,2058.90,2072.36,2046.04,2058.20
2015-01-05,2054.44,2054.44,2017.34,2020.58
2015-01-06,2022.15,2030.25,1992.44,2002.61
2015-01-07,2005.55,2029.61,2005.55,2025.90
2015-01-08,2030.61,2064.08,2030.61,2062.14
...,...,...,...,...
2022-01-03,4778.14,4796.64,4758.17,4796.56
2022-01-04,4804.51,4818.62,4774.27,4793.54
2022-01-05,4787.99,4797.70,4699.44,4700.58


In [8]:
X = df.iloc[:-1, :]
y = pd.DataFrame(df["CLOSE"][1:])
mm = MinMaxScaler(feature_range=(0,1))
ss = StandardScaler()

X_ss = ss.fit_transform(X)
y_mm =mm.fit_transform(y)

X_batches = []
y_batches = []
for i in range(60, len(X_ss)):
    X_batches.append(X_ss[i-60:i])
    y_batches.append(y_mm[i-1, :])

X_batches = np.array(X_batches)
y_batches = np.array(y_batches)

X_train = X_batches[:1500, :]
X_test = X_batches[1500:, :]

y_train = y_batches[:1500, :]
y_test = y_batches[1500:, :]

print("Training Shape: ", X_train.shape, y_train.shape)
print("Testing Shape: ", X_test.shape, y_test.shape) 

X_train_tensors = Variable(torch.Tensor(X_train))
X_test_tensors = Variable(torch.Tensor(X_test))

y_train_tensors = Variable(torch.Tensor(y_train))
y_test_tensors = Variable(torch.Tensor(y_test)) 

X_train_tensors_final = torch.reshape(X_train_tensors, (X_train_tensors.shape[0], 1, X_train_tensors.shape[1], X_train_tensors.shape[2]))
X_test_tensors_final = torch.reshape(X_test_tensors, (X_test_tensors.shape[0], 1, X_test_tensors.shape[1], X_test_tensors.shape[2]))

print("Training Shape: ", X_train_tensors_final.shape, y_train_tensors.shape)
print("Testing Shape: ", X_test_tensors_final.shape, y_test_tensors.shape) 

Training Shape:  (1500, 60, 4) (1500, 1)
Testing Shape:  (207, 60, 4) (207, 1)
Training Shape:  torch.Size([1500, 1, 60, 4]) torch.Size([1500, 1])
Testing Shape:  torch.Size([207, 1, 60, 4]) torch.Size([207, 1])


In [9]:
class Netz(nn.Module):
    def __init__(self, outputs, inputs, hiddens, layers, seq_length):
        super(Netz, self).__init__()
        self.outputs = outputs
        self.inputs = inputs
        self.hiddens = hiddens
        self.layers = layers
        self.seq_length = seq_length

        self.lstm1 = nn.LSTM(input_size=inputs, hidden_size=hiddens, num_layers=layers, batch_first=True)
        self.fc1 = nn.Linear(hiddens, 128)
        self.fc2 = nn.Linear(128, outputs)
        self.relu = nn.ReLU()
        

    def forward(self, x):
        #print("x: ", x.shape)
        h_0 = self.initHiddenInternal(x)
        c_0 = self.initHiddenInternal(x)
        output, (hn, cn) = self.lstm1(x, (h_0, c_0))
        output = output.view(-1, self.hiddens)    
        out = self.relu(output)
        out = self.fc1(out)
        out = self.relu(out)
        out = self.fc2(out)
        return out
    
    def initHiddenInternal(self, x):
        return Variable(torch.zeros(self.layers, x.size(0), self.hiddens))

Als Output-shape kriege ich: torch.Size([60, 1]).

Diese passt halt nicht zur target size von: torch.Size([1])

In [10]:
num_epochs = 10
learning_rate = 0.0005

inputs = 4
hiddens = 6
layers = 2
outputs = 1

model = Netz(outputs, inputs, hiddens, layers, X_train_tensors_final.shape[1])